# Amazon Lab 2
Paine(HE Peiwei, 54471962)
## Research Question: the most controversial digital music product
First, open the raw data of Digital Music on Amazon, and create lists for reviews and products seperately, and save the data as pickles.

In [1]:
import json
import pandas as pd
fn = "Digital_Music_5.json"
reviews = (eval(line) for line in open(fn))
reviews = pd.DataFrame(reviews)

In [2]:
fn = "meta_Digital_Music.json"
asins = set(reviews.asin.unique())
products = (eval(line) for line in open(fn))
products = (p for p in products if p['asin'] in asins)
products = pd.DataFrame(products)
products = products.set_index("asin")

In [3]:
products.to_pickle("products.pickle")
reviews.to_pickle("reviews.pickle")

Second, calculate the mean review amount and score per product, using the groupby function to put the data in correct order.

In [4]:
reviews = pd.read_pickle("reviews.pickle")
counts = reviews.asin.value_counts()

In [5]:
reviews.overall[reviews.asin == counts.index[0]].mean()
reviews.overall[reviews.asin == counts.index[0]].value_counts()
stats = reviews.groupby(['asin'])['overall'].agg(['count', 'mean'])

Third, calculate the number of good and bad reviews seperately, by using value_count according to the score. Fill the missing value with 0. And combine the 2 series into 1 dataframe, and get the min value of number of good or bad reviews. These steps are doing the same thing as the last lab, that define a function called "Controversiality".

In [6]:
good = reviews.asin[reviews.overall == 5].value_counts()
bad = reviews.asin[reviews.overall <= 2].value_counts()
c = good.combine(bad, func=min, fill_value=0)
c = c.sort_values(ascending=False)
c.head()

B000084T18    75
B0007NFL18    48
B00008H2LB    44
B00006690F    39
B00005YW4H    31
Name: asin, dtype: int64

In [7]:
products = pd.read_pickle("products.pickle")
products['controversiality'] = c
products = products.sort_values("controversiality", ascending=False)

Fourth, get the top 5 most controversial products, and manage the dataframe, and present the useful data only, choosing by columns.

In [8]:
products = products.join(stats)
products.head()

,brand,categories,description,imUrl,price,related,salesRank,title,controversiality,count,mean
asin,,,,,,,,,,,
B000084T18,,"[[CDs & Vinyl, Pop], [CDs & Vinyl, Rap & Hip-H...",He&#x92;s been shot nine times. Incarcerated. ...,http://ecx.images-amazon.com/images/I/61AYpcmB...,8.15,"{'buy_after_viewing': ['B0007NFL18', 'B00000I5...",{'Music': 2573},Get Rich Or Die Tryin,75.0,271,3.468635
B0007NFL18,Umgd/Aftermath,"[[CDs & Vinyl, Pop], [CDs & Vinyl, Rap & Hip-H...",50 CENT THE MASSACRE,http://ecx.images-amazon.com/images/I/51QDFS68...,10.98,"{'buy_after_viewing': ['B000084T18', 'B000BEZQ...",{'Music': 10505},The Massacre,48.0,272,2.558824
B00008H2LB,NaN,"[[CDs & Vinyl, Alternative Rock, American Alte...",Linkin Park&#x92;s second studio effort (not c...,http://ecx.images-amazon.com/images/I/31N10AYD...,4.99,"{'buy_after_viewing': ['B00004Z459', 'B00K03VZ...",{'Music': 450},Meteora,44.0,152,3.467105
B00006690F,NaN,"[[CDs & Vinyl, Pop], [CDs & Vinyl, Rap & Hip-H...",Any lingering doubts as to the depth of Eminem...,http://ecx.images-amazon.com/images/I/410VJKV7...,9.49,"{'buy_after_viewing': ['B00FKAA3MA', 'B00004T9...",{'Music': 62107},The Eminem Show [Limited Edition w/ Bonus DVD],39.0,204,3.911765
B00005YW4H,Emm/Blue Note,"[[CDs & Vinyl, Broadway & Vocalists], [CDs & V...",It is not just the timbre of Norah Jones's voi...,http://ecx.images-amazon.com/images/I/51z3tmwT...,9.95,"{'buy_after_viewing': ['B004EBT5CU', 'B000RL22...",{'Music': 429},Come Away with Me,31.0,190,3.963158


In [9]:
products[["title"] + list(products.columns[-3:])].head()
products.ix[:, -4:].head()

,title,controversiality,count,mean
asin,,,,
B000084T18,Get Rich Or Die Tryin,75.0,271,3.468635
B0007NFL18,The Massacre,48.0,272,2.558824
B00008H2LB,Meteora,44.0,152,3.467105
B00006690F,The Eminem Show [Limited Edition w/ Bonus DVD],39.0,204,3.911765
B00005YW4H,Come Away with Me,31.0,190,3.963158


In [10]:
reviews.overall[reviews.asin == "B000084T18"].value_counts()

5.0    100
4.0     63
1.0     61
3.0     33
2.0     14
Name: overall, dtype: int64

# Result: 

According to the result, 50 Cent-Get Rich Or Die Tryin(2003) is the most controversial music product, and I check more detailed score distribution of this product,, the result is consistent with last week's lab, however, the controversiality value is different. There are 100 good reviews, 75 bad reviews, so the controversiality should be 75, this week's result is correct. However, last week's result was 48.
The reason might be :
print(controversiality(most))

I'm not sure how pure python calculate the data and give the "most", so it might give the wrong result, but last week I didn't notice that. But this week, the program combine the good and bad reviews into 1 dataframe, and what I do is to rearrange the rows and columns by the controversiality value, and finally print the correct result.

In [11]:
import json
import gzip

In [12]:
reviewers,products ={},{}
fn = "Digital_Music_5.json"
reviews = []
for line in open(fn):
    review = eval(line)
    reviews.append(review)

In [13]:
product_ids = set()
for review in reviews:
    product_id = review['asin']
    product_ids.add(product_id)
print(list(product_ids)[:5])

['B00000AG8D', 'B0009VJWQS', 'B0000072SX', 'B004W9CG5W', 'B00000064G']


In [14]:
products = {}
fn = "meta_Digital_Music.json"
for line in open(fn):
    d = eval(line)
    if d['asin'] in product_ids:
        products[d['asin']] = d
print(len(products))

3568


In [15]:
p_reviews = {}
for review in reviews:
    asin = review['asin']
    if asin not in p_reviews:
        p_reviews[asin] = [review]
    else:
        p_reviews[asin].append(review)

In [16]:
def n_reviews(asin):
    reviews = p_reviews[asin]
    return len(reviews)
ids = sorted(p_reviews, key=n_reviews, reverse=True)
most = ids[0]
reviews_most = p_reviews[most]
def count_scores(reviews):
    counts = {score: 0 for score in range(1,6)}
    for review in reviews:
        score = review['overall']
        counts[score] += 1
    return counts

In [17]:
def controversiality(asin):
    reviews = p_reviews[asin]
    counts = count_scores(reviews)
    negative = counts[1] + counts[2]
    positive = counts[5]
    return min(negative, positive)
print(controversiality(most))

48


In [18]:
ids = sorted(p_reviews, key=controversiality, reverse=True)
controv = ids[0]
print("Most controversial:", controv, "(", products[controv]['title'],"): ", controversiality(most))

Most controversial: B000084T18 ( Get Rich Or Die Tryin ):  48


In [19]:
%timeit -n 10 controversiality

10 loops, best of 3: 97.7 ns per loop


In [20]:
%timeit -n 10 good.combine(bad, func=min, fill_value=0)

10 loops, best of 3: 334 ms per loop


# Comparision

I compare 2 functions by using %timeit. Controversiality is 3:97,7 ns per loop, but good.combine function is 3:334 ms per loop. It seems controversiality is faster than good.combine function, but it takes long time to build dictionaries for reviews and products. So I prefer the combine funtion in pandas.